# Serialise Detections to a JSON File

---

[![Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/roboflow/supervision/blob/develop/docs/notebooks/detections-to-jsonsink.ipynb)

This cookbook introduce [sv.JSONSink](https://supervision.roboflow.com/develop/detection/tools/save_detections/#supervision.detection.tools.json_sink.JSONSink) tool designed to write captured object detection data to file from video streams/file

Click the `Open in Colab` button to run the cookbook on Google Colab.

In [ ]:
!pip install -q inference requests tqdm  supervision==0.19.0rc6

In [ ]:
import supervision as sv
from supervision.assets import download_assets, VideoAssets
from inference import InferencePipeline
from inference.core.interfaces.camera.entities import VideoFrame
import numpy as np
import pandas as pd

In [ ]:
SOURCE_VIDEO_PATH = download_assets(VideoAssets.PEOPLE_WALKING)
CONFIDENCE_THRESHOLD = 0.3
IOU_THRESHOLD = 0.7

## Initialize ByteTrack

In [20]:
byte_track = sv.ByteTrack()

## Initialize JsonSink

In [21]:
json_sink = sv.JSONSink('detections.json')
json_sink.open()

## Process video to class to track detections and save detections to json file

All the operations we plan to perform for each frame of our video - detection, tracking, annotation, and write to json - are encapsulated in a function named `callback`.

In [22]:
def callback(predictions: dict, frame: VideoFrame) -> np.ndarray:
    detections = sv.Detections.from_inference(predictions)
    detections = byte_track.update_with_detections(detections)
    json_sink.append(detections, custom_data={'frame_number': frame.frame_id})
    print(f"Processed Frame ID: {frame.frame_id}")

In [23]:
#Aliases: https://inference.roboflow.com/reference_pages/model_aliases/

REGISTERED_ALIASES = {
    "yolov8n-640": "coco/3",
    "yolov8n-1280": "coco/9",
    "yolov8m-640": "coco/8",
    "yolov8x-1280": "coco/10",
}

def resolve_roboflow_model_alias(model_id: str) -> str:
    return REGISTERED_ALIASES.get(model_id, model_id)

alias = "yolov8n-640"
model_name = resolve_roboflow_model_alias(alias)

pipeline = InferencePipeline.init(
    model_id=model_name,
    video_reference=SOURCE_VIDEO_PATH,
    on_prediction=callback,
    iou_threshold=IOU_THRESHOLD,
    confidence_threshold=CONFIDENCE_THRESHOLD,
)

In [ ]:
pipeline.start()
pipeline.join()

In [ ]:
# Close JSONSink
json_sink.write_and_close()

## Visualizate results of detections json data with Pandas

Let's take a look at our resulting data with by using Pandas.

It will also be created in your current directory with the name detections.json as well.

In [14]:
df = pd.read_json('detections.json')
df

,x_min,y_min,x_max,y_max,class_id,confidence,tracker_id,frame_number
0,1460.000000,469.000000,1544.000000,641.000000,0,0.820143,1,1
1,1447.000000,320.000000,1516.000000,480.000000,0,0.791599,2,1
2,259.000000,435.000000,326.000000,610.000000,0,0.775027,3,1
3,1142.000000,950.000000,1245.000000,1080.000000,0,0.768064,4,1
4,665.000000,649.000000,744.000000,851.000000,0,0.762994,5,1
...,...,...,...,...,...,...,...,...
15824,1854.766602,259.925568,1914.519043,442.587036,0,0.715025,278,341
15825,391.996429,470.574280,460.881470,632.927673,0,0.506562,274,341
15826,74.250565,700.197937,173.041855,893.159363,0,0.477815,262,341
15827,1012.475769,170.831772,1058.312744,297.397980,0,0.565866,239,341
